<a href="https://colab.research.google.com/gist/Leen-Alsharbaji/a657f5e8eb4c744005ac30c6e9b0550f/copy-of-unityassistant1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading the model using unsloth

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math. Coding verisons!
    model_name = "unsloth/Qwen2.5-7B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Qwen2 patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

using Lora adapter to make the model more efficient

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

trying out the current abilities of the model

In [ ]:
# Define your test input
user_input = "give me a c# code that sums two values A and B"

# Tokenize the input
input_ids = tokenizer.encode(user_input, return_tensors="pt").to(model.device)

# Generate response
with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        max_length=512,  # Adjust the max length as needed
        temperature=0.7,  # Controls randomness
        top_p=0.9,        # Controls diversity
        repetition_penalty=1.2,  # Penalizes repetitive phrases
        do_sample=True     # Enables sampling for diverse responses
    )

# Decode and print the response
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(f"You: {user_input}")
print(f"Model: {response}")

You: give me a c# code that sums two values A and B
Model: give me a c# code that sums two values A and B, where the result should be 10 if either of them is zero. Here's a simple C# code snippet that sums two values `A` and `B`, but ensures that the result is always 10 if either of the values is zero:

```csharp
using System;

class Program
{
    static void Main()
    {
        int A = 5;
        int B = 0; // You can change these values to test different scenarios

        int sum = SumWithZeroCheck(A, B);
        
        Console.WriteLine($"The sum is: {sum}");
    }

    static int SumWithZeroCheck(int A, int B)
    {
        return (A == 0 || B == 0) ? 10 : A + B;
    }
}
```

### Explanation:
- The function `SumWithZeroCheck` takes two integer parameters `A` and `B`.
- It checks whether either `A` or `B` is equal to zero.
- If one of them is zero, it returns 10 directly.
- Otherwise, it calculates the sum of `A` and `B`.

You can modify the input values (`A` and `B`) in the `Ma

Testing out the model with Unity with progressively harder/detailed tasks (it generated satisfacotry results)

In [ ]:
# Define your test input
user_input = "how can i write a function to move a character in unity using c#"

# Tokenize the input
input_ids = tokenizer.encode(user_input, return_tensors="pt").to(model.device)

# Generate response
with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        max_length=512,  # Adjust the max length as needed
        temperature=0.7,  # Controls randomness
        top_p=0.9,        # Controls diversity
        repetition_penalty=1.2,  # Penalizes repetitive phrases
        do_sample=True     # Enables sampling for diverse responses
    )

# Decode and print the response
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(f"You: {user_input}")
print(f"Model: {response}")

You: how can i write a function to move a character in unity using c#
Model: how can i write a function to move a character in unity using c#?
To create a simple movement script for your Unity game object (character) in C#, you'll need to use the `Input` class and manipulate its position. Here's how you could implement this:

1. **Create a new Script**: In Unity, go to `Assets > Create > C# Script`. Name it something like "PlayerMovement".

2. **Edit the Script**:
   - Open the newly created `.cs` file.
   - Add the necessary code inside.

Here’s an example of such a basic player movement system that moves a 3D GameObject forward based on arrow key input or WASD keys:

```csharp
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class PlayerMovement : MonoBehaviour
{
    // Movement speed variable
    public float speed = 5f; 

    void Update()
    {
        Move();
    }

    private void Move()
    {
        // Get horizontal and vertical inputs f

In [ ]:
# Define your test input
user_input = "How can I create a movement system in Unity where the character can walk smoothly on slopes, without sliding or moving too fast uphill?"

# Tokenize the input
input_ids = tokenizer.encode(user_input, return_tensors="pt").to(model.device)

# Generate response
with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        max_length=512,  # Adjust the max length as needed
        temperature=0.7,  # Controls randomness
        top_p=0.9,        # Controls diversity
        repetition_penalty=1.2,  # Penalizes repetitive phrases
        do_sample=True     # Enables sampling for diverse responses
    )

# Decode and print the response
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(f"You: {user_input}")
print(f"Model: {response}")

You: How can I create a movement system in Unity where the character can walk smoothly on slopes, without sliding or moving too fast uphill?
Model: How can I create a movement system in Unity where the character can walk smoothly on slopes, without sliding or moving too fast uphill? To achieve smooth walking on slopes and prevent excessive speed when going uphill, you need to implement a custom movement script that takes into account slope angles. Here’s how you can do it:

1. **Create a Script for Character Movement**:
   - Create a new C# script named `SmoothMovement.cs`.
   - Attach this script to your player GameObject.

```csharp
using UnityEngine;

public class SmoothMovement : MonoBehaviour
{
    public float moveSpeed = 5f;
    private Rigidbody rb;

    void Start()
    {
        rb = GetComponent<Rigidbody>();
    }

    // Update is called once per frame
    void FixedUpdate()
    {
        HandleInput();
    }

    void HandleInput()
    {
        Vector3 movementDirection 

In [ ]:
# Define your test input
user_input = "give me code to make a monster follow a player using Navmesh in unity c#"

# Tokenize the input
input_ids = tokenizer.encode(user_input, return_tensors="pt").to(model.device)

# Generate response
with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        max_length=512,  # Adjust the max length as needed
        temperature=0.7,  # Controls randomness
        top_p=0.9,        # Controls diversity
        repetition_penalty=1.2,  # Penalizes repetitive phrases
        do_sample=True     # Enables sampling for diverse responses
    )

# Decode and print the response
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(f"You: {user_input}")
print(f"Model: {response}")

You: give me code to make a monster follow a player using Navmesh in unity c#
Model: give me code to make a monster follow a player using Navmesh in unity c# Here's an example of how you can create a basic AI system for your monster to chase the player using Unity's Navigation System and C#. This script assumes that you have already set up a NavMesh Agent on both the Monster (enemy) and Player GameObjects.

1. Create a new C# Script named `MonsterChaseAI` or any other name you prefer.
2. Attach this script to the enemy GameObject.

```csharp
using UnityEngine;
using UnityEngine.AI;

public class MonsterChaseAI : MonoBehaviour
{
    public Transform target; // Reference to the player

    private NavMeshAgent agent;

    void Start()
    {
        if (!target)
            Debug.LogError("Player not assigned!");

        agent = GetComponent<NavMeshAgent>();
    }

    void Update()
    {
        if(target != null && agent.enabled)
        {
            Vector3 directionToTarget = target

In [ ]:
# Define your test input
user_input = "write me a code to show the basic unity lifecycle "

# Tokenize the input
input_ids = tokenizer.encode(user_input, return_tensors="pt").to(model.device)

# Generate response
with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        max_length=512,  # Adjust the max length as needed
        temperature=0.7,  # Controls randomness
        top_p=0.9,        # Controls diversity
        repetition_penalty=1.2,  # Penalizes repetitive phrases
        do_sample=True     # Enables sampling for diverse responses
    )

# Decode and print the response
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(f"You: {user_input}")
print(f"Model: {response}")

You: write me a code to show the basic unity lifecycle 
Model: write me a code to show the basic unity lifecycle  ```csharp
using UnityEngine;

public class BasicUnityLifecycle : MonoBehaviour
{
    void OnEnable()
    {
        Debug.Log("OnEnable called.");
    }

    void Start()
    {
        Debug.Log("Start called.");
    }

    void Update()
    {
        // Keep this for demonstration purposes, but in practice you should not rely on it.
        if (Input.GetKeyDown(KeyCode.Space))
        {
            Destroy(gameObject);
        }
    }

    void OnDestroy()
    {
        Debug.Log("Destroying object. OnDestroy is being called.");
    }
}
``` To demonstrate the Unity lifecycle of an object using C#, let's create and destroy objects at various points during their lifetime within a script attached to GameObjects.

Here’s how we can do that:

1. Create a `BasicUnityLifecycle` component with methods corresponding to different stages of its lifecycle: `OnEnable`, `Start`, etc.
2. 

In [ ]:
# Define your test input
user_input = "whats wrong with my code? using UnityEngine;using UnityEngine; using UnityEngine.AI; public class NPCFollowPlayer : MonoBehaviour { public Transform player; public float followDistance = 5f; private NavMeshAgent navMeshAgent; void Start() { navMeshAgent = GetComponent<NavMeshAgent>(); } void Update() { if (player != null && Vector3.Distance(transform.position, player.position) < followDistance) navMeshAgent.ResetPath(); else if (player != null) navMeshAgent.SetDestination(player.position); } }"



# Tokenize the input
input_ids = tokenizer.encode(user_input, return_tensors="pt").to(model.device)

# Generate response
with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        max_length=512,  # Adjust the max length as needed
        temperature=0.7,  # Controls randomness
        top_p=0.9,        # Controls diversity
        repetition_penalty=1.2,  # Penalizes repetitive phrases
        do_sample=True     # Enables sampling for diverse responses
    )

# Decode and print the response
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(f"You: {user_input}")
print(f"Model: {response}")

You: whats wrong with my code? using UnityEngine;using UnityEngine; using UnityEngine.AI; public class NPCFollowPlayer : MonoBehaviour { public Transform player; public float followDistance = 5f; private NavMeshAgent navMeshAgent; void Start() { navMeshAgent = GetComponent<NavMeshAgent>(); } void Update() { if (player != null && Vector3.Distance(transform.position, player.position) < followDistance) navMeshAgent.ResetPath(); else if (player != null) navMeshAgent.SetDestination(player.position); } }
Model: whats wrong with my code? using UnityEngine;using UnityEngine; using UnityEngine.AI; public class NPCFollowPlayer : MonoBehaviour { public Transform player; public float followDistance = 5f; private NavMeshAgent navMeshAgent; void Start() { navMeshAgent = GetComponent<NavMeshAgent>(); } void Update() { if (player != null && Vector3.Distance(transform.position, player.position) < followDistance) navMeshAgent.ResetPath(); else if (player != null) navMeshAgent.SetDestination(player.posit

In [ ]:
# Define your test input
user_input = "explain the unity physics to me how does it work?"

# Tokenize the input
input_ids = tokenizer.encode(user_input, return_tensors="pt").to(model.device)

# Generate response
with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        max_length=512,  # Adjust the max length as needed
        temperature=0.7,  # Controls randomness
        top_p=0.9,        # Controls diversity
        repetition_penalty=1.2,  # Penalizes repetitive phrases
        do_sample=True     # Enables sampling for diverse responses
    )

# Decode and print the response
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(f"You: {user_input}")
print(f"Model: {response}")

You: explain the unity physics to me how does it work?
Model: explain the unity physics to me how does it work? Unity's Physics System is a powerful tool for simulating real-world physical behavior in games and interactive applications. It allows developers to create realistic interactions between objects, such as collisions, gravity, friction, and more. Here’s an overview of how Unity’s physics system works:

### Key Components

1. **Physics Engine**:
   - The core engine handles all calculations related to motion, forces, and collision detection.
   
2. **Rigidbody Component**: 
   - This component enables a GameObject to participate in the physics simulation by applying forces (like gravity) and handling collisions.

3. **Collider Component**:  
   - Detects when two or more rigidbodies come into contact with each other. Colliders can be different shapes like boxes, spheres, capsules, etc., depending on your needs.

4. **Force Application**:
   - Forces are applied through script co

In [ ]:
# Define your test input
user_input = "can you generate images?"

# Tokenize the input
input_ids = tokenizer.encode(user_input, return_tensors="pt").to(model.device)

# Generate response
with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        max_length=512,  # Adjust the max length as needed
        temperature=0.7,  # Controls randomness
        top_p=0.9,        # Controls diversity
        repetition_penalty=1.2,  # Penalizes repetitive phrases
        do_sample=True     # Enables sampling for diverse responses
    )

# Decode and print the response
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(f"You: {user_input}")
print(f"Model: {response}")


You: can you generate images?
Model: can you generate images? I'm sorry, but as a text-based AI assistant, I don't have the capability to create or generate images. However, if you'd like help with something related to image creation (such as generating ideas for an artist, writing descriptions of what you want in an image, finding tools that can generate images from text, etc.), feel free to ask and I'll do my best to assist! 😊

If you need more specific assistance regarding image generation using software/tools, let me know which one you're interested in (like DALL-E, Midjourney, Stable Diffusion, etc.) and I can provide guidance on how to use them effectively. 🌟


**Fine-Tuning the Model**




In [ ]:
!pip install datasets transformers

In [ ]:
from datasets import load_dataset

# Load CodeSearchNet dataset correctly
dataset = load_dataset("code_search_net", split="train")

# Filter the dataset to only include HTML, CSS, and JavaScript code
relevant_languages = ["html", "css", "javascript"]
filtered_dataset = dataset.filter(lambda x: x['language'] in relevant_languages)

# Show a sample from the filtered dataset
print(filtered_dataset[0])



README.md:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

code_search_net.py:   0%|          | 0.00/8.44k [00:00<?, ?B/s]

The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


python.zip:   0%|          | 0.00/941M [00:00<?, ?B/s]

java.zip:   0%|          | 0.00/1.06G [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/1880853 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100529 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/89154 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1880853 [00:00<?, ? examples/s]

{'repository_name': 'j-/ok', 'func_path_in_repository': 'ok.js', 'func_name': 'Map', 'whole_func_string': 'function Map (properties) {\n\t\tok.Data.apply(this, arguments);\n\t\tvar defaults = this.getDefaults();\n\t\tif (!this.properties) {\n\t\t\tthis.properties = {};\n\t\t}\n\t\tif (defaults) {\n\t\t\tthis.initProperties(defaults);\n\t\t}\n\t\tif (properties) {\n\t\t\tthis.setMap(properties);\n\t\t}\n\t}', 'language': 'javascript', 'func_code_string': 'function Map (properties) {\n\t\tok.Data.apply(this, arguments);\n\t\tvar defaults = this.getDefaults();\n\t\tif (!this.properties) {\n\t\t\tthis.properties = {};\n\t\t}\n\t\tif (defaults) {\n\t\t\tthis.initProperties(defaults);\n\t\t}\n\t\tif (properties) {\n\t\t\tthis.setMap(properties);\n\t\t}\n\t}', 'func_code_tokens': ['function', 'Map', '(', 'properties', ')', '{', 'ok', '.', 'Data', '.', 'apply', '(', 'this', ',', 'arguments', ')', ';', 'var', 'defaults', '=', 'this', '.', 'getDefaults', '(', ')', ';', 'if', '(', '!', 'this', '.

In [ ]:
print(filtered_dataset.column_names)


['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url']


In [ ]:
# Define a custom prompt formatting function
def formatting_prompts_func(examples):
    inputs = examples['func_code_string']  # Use the 'func_code_string' column for the code snippets
    instructions = ["Complete the following code snippet:" for _ in inputs]

    # Format the dataset into the desired structure
    texts = [f"### Instruction: {instruction}\n### Input: {input}\n### Response:"
             for instruction, input in zip(instructions, inputs)]

    return {"text": texts}

# Apply the formatting function to the dataset
formatted_dataset = filtered_dataset.map(formatting_prompts_func, batched=True)

# Show a sample of the formatted dataset
print(formatted_dataset[0])


Map:   0%|          | 0/123889 [00:00<?, ? examples/s]

{'repository_name': 'j-/ok', 'func_path_in_repository': 'ok.js', 'func_name': 'Map', 'whole_func_string': 'function Map (properties) {\n\t\tok.Data.apply(this, arguments);\n\t\tvar defaults = this.getDefaults();\n\t\tif (!this.properties) {\n\t\t\tthis.properties = {};\n\t\t}\n\t\tif (defaults) {\n\t\t\tthis.initProperties(defaults);\n\t\t}\n\t\tif (properties) {\n\t\t\tthis.setMap(properties);\n\t\t}\n\t}', 'language': 'javascript', 'func_code_string': 'function Map (properties) {\n\t\tok.Data.apply(this, arguments);\n\t\tvar defaults = this.getDefaults();\n\t\tif (!this.properties) {\n\t\t\tthis.properties = {};\n\t\t}\n\t\tif (defaults) {\n\t\t\tthis.initProperties(defaults);\n\t\t}\n\t\tif (properties) {\n\t\t\tthis.setMap(properties);\n\t\t}\n\t}', 'func_code_tokens': ['function', 'Map', '(', 'properties', ')', '{', 'ok', '.', 'Data', '.', 'apply', '(', 'this', ',', 'arguments', ')', ';', 'var', 'defaults', '=', 'this', '.', 'getDefaults', '(', ')', ';', 'if', '(', '!', 'this', '.

In [ ]:
def formatting_prompts_func(examples):
    inputs = examples['func_code_string']  # The column containing the code snippets
    instructions = ["Complete the following code snippet:" for _ in inputs]

    # Format the dataset into the desired structure
    texts = [f"### Instruction: {instruction}\n### Input: {input}\n### Response:"
             for instruction, input in zip(instructions, inputs)]

    return {"text": texts}

# Apply the formatting function to the dataset
formatted_dataset = filtered_dataset.map(formatting_prompts_func, batched=True)

# Show a sample of the formatted dataset
print(formatted_dataset[0])


{'repository_name': 'j-/ok', 'func_path_in_repository': 'ok.js', 'func_name': 'Map', 'whole_func_string': 'function Map (properties) {\n\t\tok.Data.apply(this, arguments);\n\t\tvar defaults = this.getDefaults();\n\t\tif (!this.properties) {\n\t\t\tthis.properties = {};\n\t\t}\n\t\tif (defaults) {\n\t\t\tthis.initProperties(defaults);\n\t\t}\n\t\tif (properties) {\n\t\t\tthis.setMap(properties);\n\t\t}\n\t}', 'language': 'javascript', 'func_code_string': 'function Map (properties) {\n\t\tok.Data.apply(this, arguments);\n\t\tvar defaults = this.getDefaults();\n\t\tif (!this.properties) {\n\t\t\tthis.properties = {};\n\t\t}\n\t\tif (defaults) {\n\t\t\tthis.initProperties(defaults);\n\t\t}\n\t\tif (properties) {\n\t\t\tthis.setMap(properties);\n\t\t}\n\t}', 'func_code_tokens': ['function', 'Map', '(', 'properties', ')', '{', 'ok', '.', 'Data', '.', 'apply', '(', 'this', ',', 'arguments', ')', ';', 'var', 'defaults', '=', 'this', '.', 'getDefaults', '(', ')', ';', 'if', '(', '!', 'this', '.

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # For faster training with shorter sequences
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",  # Use this for WandB etc.
    ),
)

Map (num_proc=2):   0%|          | 0/123889 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 123,889 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 10
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000
10,0.000000


TrainOutput(global_step=10, training_loss=0.0, metrics={'train_runtime': 74.8626, 'train_samples_per_second': 1.069, 'train_steps_per_second': 0.134, 'total_flos': 1115202231822336.0, 'train_loss': 0.0, 'epoch': 0.0006457341189765114})

In [ ]:
import os
print(os.getcwd())

model.save_pretrained("/content/drive/MyDrive/UnityVS-V1")
tokenizer.save_pretrained("/content/drive/MyDrive/UnityVS-V1")

/content


('/content/drive/MyDrive/UnityVS-V1/tokenizer_config.json',
 '/content/drive/MyDrive/UnityVS-V1/special_tokens_map.json',
 '/content/drive/MyDrive/UnityVS-V1/vocab.json',
 '/content/drive/MyDrive/UnityVS-V1/merges.txt',
 '/content/drive/MyDrive/UnityVS-V1/added_tokens.json',
 '/content/drive/MyDrive/UnityVS-V1/tokenizer.json')

In [ ]:
print(type(model))

<class 'tuple'>


In [ ]:


# Now save the model and tokenizer
model.save_pretrained("/content/drive/MyDrive/UnityVS-V2")
tokenizer.save_pretrained("/content/drive/MyDrive/UnityVS-V2")

KeyboardInterrupt: 

In [ ]:
#if True:
 #   model.push_to_hub_gguf(
  #      "LinCorbaci/GhostCoder",  # Replace with your username and model name
   #     tokenizer=tokenizer,
    #    quantization_method="q4_k_m",  # Change to q4_k_m quantization method
     #   token=""  # Replace with your token
    #)

model.save_pretrained_gguf("/content/drive/MyDrive/UnityVS-V2", tokenizer, quantization_method="q4_k_m")


Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.5G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 2.84 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


  4%|▎         | 1/28 [00:00<00:04,  6.61it/s]
We will save to Disk and not RAM now.
100%|██████████| 28/28 [05:58<00:00, 12.81s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving /content/drive/MyDrive/UnityVS-V2/pytorch_model-00001-of-00004.bin...
Unsloth: Saving /content/drive/MyDrive/UnityVS-V2/pytorch_model-00002-of-00004.bin...
Unsloth: Saving /content/drive/MyDrive/UnityVS-V2/pytorch_model-00003-of-00004.bin...
Unsloth: Saving /content/drive/MyDrive/UnityVS-V2/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting qwen2 model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at /content/drive/MyDrive/UnityVS-V2 into f16 GGUF format.
The output location will be /content/drive/MyDrive/UnityVS-V2/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: UnityVS-V2
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin'
INFO:hf-to-gguf:token_embd.

In [ ]:
import os
file_path = "/content/drive/MyDrive/UnityVS-V2/unsloth.Q4_K_M.gguf"
print(f"Does the file exist? {os.path.exists(file_path)}")

Does the file exist? True


In [ ]:
!pip freeze > requirements.txt

In [ ]:
from google.colab import files
files.download('requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
print("Current Working Directory:", os.getcwd())

Current Working Directory: /content


In [ ]:
import os
folder_path = "/content/drive/MyDrive/UnityVS-V2"
print(f"Files in {folder_path}:")
for file in os.listdir(folder_path):
    print(file)

Files in /content/drive/MyDrive/UnityVS-V2:
tokenizer_config.json
special_tokens_map.json
added_tokens.json
vocab.json
merges.txt
tokenizer.json
config.json
generation_config.json
pytorch_model-00004-of-00004.bin
pytorch_model.bin.index.json
